In [28]:
import sys
import findspark
findspark.init()
from pyspark import SparkContext,SparkConf

In [2]:
configuration=SparkConf().setMaster("local[*]").setAppName("Recommendation System")

In [3]:
sc=SparkContext(conf=configuration)

In [15]:
source_file=sc.textFile("C://Users//kk//OneDrive//Desktop//ml-100k//u.data")

In [75]:
def movie_names():
    movie_names={}
    with open("C://Users//kk//OneDrive//Desktop//ml-100k//u.ITEM") as f:
        lines=f.readlines()
        for line in lines:
            elements=line.split('|')
            movie_names[int(elements[0])]=elements[1]
        return movie_names
def  filter_duplicates(x):
    user_ratings=x[1]
    (movie1,rating1)=user_ratings[0]
    (movie2,rating2)=user_ratings[1]
    return movie1 < movie2
def remove_user_id(x):
    user_ratings=x[1]
    (movie1,rating1)=user_ratings[0]
    (movie2,rating2)=user_ratings[1]
    return ((movie1,movie2),(rating1,rating2))

In [55]:
import math
from math import sqrt
def cosine_similarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [56]:
modified_source_file=source_file.map(lambda x:x.split('\t')).map(lambda x:(int(x[0]),(int(x[1]),int(x[2]))))

In [57]:
joined_file=modified_source_file.join(modified_source_file)

In [58]:
filtered_joined_file=joined_file.filter(filter_duplicates)

In [59]:
modifeid_mvies=filtered_joined_file.map(remove_user_id)

In [60]:
movies=modifeid_mvies.groupByKey()

In [61]:
movie_pair_similarities=movies.mapValues(cosine_similarity).cache()

In [62]:
movie_id=50

In [64]:
score_threshold=0.97
co_occurence=50
filtered_result=movie_pair_similarities.filter(lambda x:(x[0][0]==movie_id or x[0][1]==movie_id) and \
                                                  x[1][0] > score_threshold and x[1][1]>co_occurence)

In [68]:
moviedict=movie_names()

In [73]:
filtered_result=filtered_result.map(lambda x : (x[1],x[0])).sortByKey(ascending=False).take(10)

In [74]:
print("Top 5 Similar movies for ", moviedict[movie_id])
for i1 in filtered_result:
    (score,pair)=i1
    similar_movie=pair[0]
    if similar_movie==movie_id:
        similar_movie=pair[1]
    print(moviedict[similar_movie] + "\t Score" + str(score[0]) + "\t Strength "+ str(score[1]))

Top 10 Similar movies for  Star Wars (1977)
Empire Strikes Back, The (1980)	 Score0.9895522078385338	 Strength 345
Empire Strikes Back, The (1980)	 Score0.9895522078385338	 Strength 345
Return of the Jedi (1983)	 Score0.9857230861253026	 Strength 480
Return of the Jedi (1983)	 Score0.9857230861253026	 Strength 480
Raiders of the Lost Ark (1981)	 Score0.981760098872619	 Strength 380
Raiders of the Lost Ark (1981)	 Score0.981760098872619	 Strength 380
20,000 Leagues Under the Sea (1954)	 Score0.9789385605497993	 Strength 68
20,000 Leagues Under the Sea (1954)	 Score0.9789385605497993	 Strength 68
12 Angry Men (1957)	 Score0.9776576120448436	 Strength 109
12 Angry Men (1957)	 Score0.9776576120448436	 Strength 109
